In [2]:
# main.py

import polars as pl
import os

print(os.getcwd())

from src.futuapi.futu_api import FutuApi
from src.calculator.mathmatics import add_mean, add_boll

# 设置 Polars 显示的最大行数和列数
# pl.Config.set_tbl_width(1000)  # 设置表格的最大宽度，单位为字符
pl.Config.set_tbl_cols(100)  # 设置显示的最大列数
pl.Config.set_tbl_rows(10)  # 设置显示的最大行数


def trans(df):
    df = df.with_columns(
        pl.col("time_key").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S")
    )
    df = df.with_columns(
        [
            pl.col("time_key").dt.date().cast(pl.Utf8).alias("date"),
            pl.col("time_key").dt.time().cast(pl.Utf8).alias("time"),
        ]
    )
    df = df.select(["code", "close", "change_rate", "date", "time"])

    close_df = df.filter(pl.col("time") == "15:00:00")
    check_point_df = df.filter(pl.col("time") == "14:55:00")

    df = (
        close_df.join(check_point_df, on=["code", "date"])
        .with_columns(
            [
                pl.col("close_right").cast(pl.Float64).alias("check_price"),
            ]
        )
        .select(["code", "date", "check_price", "close"])
    )

    df = df.with_columns(
        [
            (
                (pl.col("check_price") - pl.col("close").shift(5))
                / pl.col("close").shift(5)
            ).alias("del_5")
        ]
    )

    df = df.with_columns(
        [
            (
                (pl.col("check_price") - pl.col("close").shift(10))
                / pl.col("close").shift(5)
            ).alias("del_10")
        ]
    )

    return df


import time
from dataclasses import dataclass


@dataclass
class Record:
    code: str
    price: float
    del_5: float
    del_10: float
    
api = FutuApi()
info = {}
try:
    secs = [
        "SH.512660",
        "SH.515880",
        "SZ.159996",
        "SH.515790",
        "SZ.159995",
        "SH.515700",
        "SZ.159647",
        "SH.512290",
        "SH.512170",
        "SH.515220",
        "SH.515210",
        "SH.512400",
        "SZ.159825",
        "SH.512800",
        "SH.516770",
        "SH.516950",
        "SZ.159745",
        "SH.512880",
        "SH.512200",
        "SH.512690",
    ]


    for sec_id in secs:
        df = api.get_minute_kline(sec_id, "2024-01-01", "2024-11-08")
        df = trans(df)
        for row in df.iter_rows(named=True):
            # print(row)
            if row["date"] not in info:
                info[row["date"]] = []
            record = Record(
                code=row["code"],
                price=row["check_price"],
                del_5=row["del_5"],
                del_10=row["del_10"],
            )
            info[row["date"]].append(record)

    # print(info["2024-11-08"])
finally:
    api.close()

/home/grasszhang/workspace/projects/gluttonous
2024-11-09 10:34:31,575 | 69011 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=13389358
Trading days saved to /data/futu/A-shares/cfg/trading_days.toml


Processing trading days: 100%|██████████| 205/205 [00:00<00:00, 3422.20it/s]


2024-11-09 10:34:33,221 | 69011 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


In [77]:
import math
def truncate_float(number):
    """
    截断浮点数到指定的小数位数，不进行四舍五入。
    
    :param number: 要截断的浮点数
    :param decimals: 小数位数
    :return: 截断后的浮点数
    """
    factor = 10.0 ** -2
    truncated = math.trunc(number * factor) / factor
    return truncated

@dataclass
class StrategyMeta:
    crash: float
    positions: list
    fund_map: dict
    tmp_crash: float
    
    def __init__(self, crash):
        self.crash = crash
        self.positions = []
        self.fund_map = {}
        self.tmp_crash = 0.0
        
    def val(self, fund_map: dict):
        result = self.crash
        for position in self.positions:
            # print(position)
            result = result +  fund_map[position['code']] * position['cnt']
        return result
        


m_init = 1e5

st1 = StrategyMeta(m_init)
st2 = StrategyMeta(m_init)
idxx = 0

for key, value in info.items():
    value = [x for x in value if x.del_10 is not None]
    if len(value) == 0:
        continue
    
    fund_map = {x.code: x.price for x in value}
    
    
    # st1 recover
    for position in st1.positions:
        st1.crash += fund_map[position['code']] * position['cnt']
    st1.crash = st1.crash * 0.9995
    st1.tmp_crash = st1.crash
    st1.positions = []
    
    # st2 recover
    st2_interval = 1
    if idxx % st2_interval == 0:
        for position in st2.positions:
            st2.crash += fund_map[position['code']] * position['cnt']
        st2.crash = st2.crash * 0.9995
        st2.tmp_crash = st2.crash
        st2.positions = []
    
    print(key, st1.val(fund_map), st2.val(fund_map))
    # print(truncate_float(crash))
    
    # st1
    sorted_value = sorted(value, key=lambda x: x.del_10, reverse=True)
    for idx in range(3, 6):
        ele = sorted_value[idx]
        # print(ele.code, ele.price, ele.del_10)
        position = {
            'code' : ele.code,
            'cnt': truncate_float(st1.tmp_crash / 3 / ele.price),
        }
        st1.crash = st1.crash - position['cnt'] * fund_map[position['code']]
        st1.positions.append(position)
        
    # st2
    sorted_value = sorted(value, key=lambda x: x.del_10, reverse=True)
    for idx in range(3, 6):
        if idxx % st2_interval != 0:
            break
        # print(ele)
        ele = sorted_value[idx]
        # print(ele.code, ele.price, ele.del_10)
        if ele.del_10 < 0.0 and ele.del_5 < 0.0:
            continue
        
        position = {
            'code' : ele.code,
            'cnt': truncate_float(st2.tmp_crash / 3 / ele.price),
        }
        # print(st2.crash, position['cnt'] * fund_map[position['code']])
        st2.crash = st2.crash - position['cnt'] * fund_map[position['code']]
        st2.positions.append(position)
        
    idxx = idxx + 1
    print(st1.positions)
    print(st2.positions)
    # print(crash, positions)
    # time.sleep(1)

2024-01-16 99950.0 99950.0
[{'code': 'SZ.159745', 'cnt': 54300.0}, {'code': 'SH.515220', 'cnt': 27700.0}, {'code': 'SH.516950', 'cnt': 36700.0}]
[{'code': 'SZ.159745', 'cnt': 54300.0}, {'code': 'SH.515220', 'cnt': 27700.0}, {'code': 'SH.516950', 'cnt': 36700.0}]
2024-01-17 98221.86450000001 98221.86450000001
[{'code': 'SH.515220', 'cnt': 27600.0}, {'code': 'SZ.159745', 'cnt': 54800.0}, {'code': 'SH.516950', 'cnt': 36500.0}]
[]
2024-01-18 97431.72426775 98172.75356775001
[{'code': 'SH.515700', 'cnt': 22000.0}, {'code': 'SZ.159745', 'cnt': 54800.0}, {'code': 'SH.516950', 'cnt': 36400.0}]
[]
2024-01-19 96612.19400561614 98123.66719096614
[{'code': 'SH.516770', 'cnt': 32700.0}, {'code': 'SZ.159996', 'cnt': 31100.0}, {'code': 'SH.512690', 'cnt': 50700.0}]
[{'code': 'SH.516770', 'cnt': 33300.0}]
2024-01-22 92771.18520861334 96011.03765737066
[{'code': 'SH.512690', 'cnt': 49800.0}, {'code': 'SZ.159996', 'cnt': 30900.0}, {'code': 'SH.516950', 'cnt': 35800.0}]
[]
2024-01-23 92722.80061600904 95